In [1]:
import requests


In [2]:
import json

In [3]:
with open('Artworks.json') as json_file:  
    data = json.load(json_file)
with open('Artists.json') as json_file2:  
    artist = json.load(json_file2)

JSONDecodeError: Unterminated string starting at: line 91521 column 17 (char 2126867)

In [4]:
data[0]

{'Title': 'Ferdinandsbrücke Project, Vienna, Austria, Elevation, preliminary version',
 'Artist': ['Otto Wagner'],
 'ConstituentID': [6210],
 'ArtistBio': ['Austrian, 1841–1918'],
 'Nationality': ['Austrian'],
 'BeginDate': [1841],
 'EndDate': [1918],
 'Gender': ['Male'],
 'Date': '1896',
 'Medium': 'Ink and cut-and-pasted painted pages on paper',
 'Dimensions': '19 1/8 x 66 1/2" (48.6 x 168.9 cm)',
 'CreditLine': 'Fractional and promised gift of Jo Carole and Ronald S. Lauder',
 'AccessionNumber': '885.1996',
 'Classification': 'Architecture',
 'Department': 'Architecture & Design',
 'DateAcquired': '1996-04-09',
 'Cataloged': 'Y',
 'ObjectID': 2,
 'URL': 'http://www.moma.org/collection/works/2',
 'ThumbnailURL': 'http://www.moma.org/media/W1siZiIsIjU5NDA1Il0sWyJwIiwiY29udmVydCIsIi1yZXNpemUgMzAweDMwMFx1MDAzZSJdXQ.jpg?sha=137b8455b1ec6167',
 'Height (cm)': 48.6,
 'Width (cm)': 168.9}

In [1]:
class unit:
    def __init__(self,image_id,artist_id,dominant_color,max_color,department,height,width):
        self.image_id = image_id  #page_index + item_index.. incase later want to re-check
        self.artist_id = artist_id
        self.dominant_color = dominant_color
        self.max_color = max_color
        self.department = department
        self.height = height
        self.width = width


In [16]:
!pip install psycopg2


  Using cached https://files.pythonhosted.org/packages/23/7e/93c325482c328619870b6cd09370f6dbe1148283daca65115cd63642e60f/psycopg2-2.8.2.tar.gz
    Complete output from command python setup.py egg_info:
    running egg_info
    creating pip-egg-info/psycopg2.egg-info
    writing dependency_links to pip-egg-info/psycopg2.egg-info/dependency_links.txt
    writing top-level names to pip-egg-info/psycopg2.egg-info/top_level.txt
    writing pip-egg-info/psycopg2.egg-info/PKG-INFO
    writing manifest file 'pip-egg-info/psycopg2.egg-info/SOURCES.txt'
    Error: b'You need to install postgresql-server-dev-X.Y for building a server-side extension or libpq-dev for building a client-side application.\n'
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-uzvbb_bj/psycopg2/
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
!pip install extcolors

  Using cached https://files.pythonhosted.org/packages/b5/9f/5b7248c3d8a0e543875a33fc88518bfc2649d3b4fc61b8d75fc8e5b90eae/extcolors-0.1.2-py3-none-any.whl
    100% |████████████████████████████████| 2.0MB 5.0MB/s ta 0:00:011


In [6]:

postgres_insert_query = """ INSERT INTO color(image_id,artist_id,dominant_color,max_color,department,height,width) VALUES (%s,%s,%s,%s,%s,%s,%s)"""
#     query = '''INSERT INTO public.test(_id,keyword,author,publisher,date,title,abstract,url) values "%s", "%s","%s","%s","%s","%s","%s","%s" ''' % (count,items[i].keyword,items[i].author,items[i].publisher,items[i].date,items[i].title,items[i].abstract,items[i].url)
record_to_insert = (0,0,'[237, 231, 233]','[237, 231, 233]','ee',2,2)
#     record_to_insert = (5, 'One Plus 6', 950)
cur.execute(postgres_insert_query, record_to_insert)
#     cur.execute(query)
conn.commit()

In [1]:
import psycopg2
conn_string =  "host='localhost' dbname='postgres' user='postgres' password='olivegpu' port='5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [5]:
import requests
import extcolors
import psycopg2
fail_list = []
results = []
count = 0
conn_string =  "host='localhost' dbname='postgres' user='postgres' password='olivegpu' port='5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
for i in range(len(data)):
    url = data[i]['ThumbnailURL']
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image_id = data[i]['ObjectID']
            file_dir = "MOMA/"+str(image_id)+".jpg"
            count+=1
            with open(file_dir, 'wb') as f:
                f.write(response.content)
            colors,pixel_count = extcolors.extract(path = file_dir,tolerance=0)
#             print('color')
            channel = min(8,len(colors))
            dominant_color = json.dumps(colors[:channel])
            max_color=json.dumps(colors[0][0])
            department = data[i]['Department']
            height =  data[i]['Height (cm)']
            width = data[i]['Width (cm)']
#             print('ci',data[i]['ConstituentID'])
            for item in data[i]['ConstituentID']:
                artist_id = item
#                 print('im here')

                postgres_insert_query = """ INSERT INTO color_new(image_id,artist_id,dominant_color,max_color,department,height,width) VALUES (%s,%s,%s,%s,%s,%s,%s)"""
        #     query = '''INSERT INTO public.test(_id,keyword,author,publisher,date,title,abstract,url) values "%s", "%s","%s","%s","%s","%s","%s","%s" ''' % (count,items[i].keyword,items[i].author,items[i].publisher,items[i].date,items[i].title,items[i].abstract,items[i].url)
                record_to_insert = (image_id,artist_id,dominant_color,max_color,department,height,width)
            #     record_to_insert = (5, 'One Plus 6', 950)
                cur.execute(postgres_insert_query, record_to_insert)
            #     cur.execute(query)
                conn.commit()
                count+=1
#                 print('imhere too')

        if count %100 == 0:
            print("100is done",count)
    except:
         fail_list.append(data[i]['AccessionNumber'])
#             print(data[i]['AccessionNumber'], "fail")


100is done 100
100is done 200
100is done 400
100is done 600
100is done 800
100is done 900
100is done 1300
100is done 1500
100is done 2100
100is done 2200
100is done 2700
100is done 2900
100is done 3200
100is done 3300
100is done 3800
100is done 3900
100is done 4000
100is done 4100
100is done 4300
100is done 4600
100is done 5500
100is done 5600
100is done 5800
100is done 6900
100is done 7100
100is done 7200
100is done 7400
100is done 7500
100is done 7900
100is done 8000
100is done 8200
100is done 8400
100is done 8500
100is done 8800
100is done 8900
100is done 9000
100is done 9300
100is done 9500
100is done 9800
100is done 9900
100is done 10000
100is done 10100
100is done 10200
100is done 10300
100is done 10500
100is done 10700
100is done 11400
100is done 11900
100is done 12100
100is done 12200
100is done 12500
100is done 12800
100is done 13000
100is done 13500
100is done 13700
100is done 14000
100is done 14100
100is done 14300
100is done 14400
100is done 14700
100is done 15100
100is don

100is done 102800
100is done 103000
100is done 103100
100is done 103200
100is done 104100
100is done 104200
100is done 104300
100is done 104400
100is done 104800
100is done 104900
100is done 105000
100is done 105100
100is done 105300
100is done 105600
100is done 106000
100is done 106100
100is done 106400
100is done 106600
100is done 106700
100is done 106800
100is done 106900
100is done 107000
100is done 107100
100is done 107200
100is done 107300
100is done 107800
100is done 108000
100is done 108500
100is done 108700
100is done 109000
100is done 109100
100is done 109400
100is done 109700
100is done 110700
100is done 110800
100is done 110900
100is done 111000
100is done 111300
100is done 111700
100is done 112200
100is done 112300
100is done 112400
100is done 112600
100is done 112800
100is done 112900
100is done 113000
100is done 113200
100is done 113600
100is done 113700
100is done 113900
100is done 114000
100is done 114100
100is done 114200
100is done 114300
100is done 114400
100is done

In [14]:
a = "/Users/glwang/Downloads/6.894/Final_project/MOMA/"+data[0]['AccessionNumber']+".jpg"

In [17]:
img_id = data[0]['AccessionNumber'].replace('.','_')
file_dir = "/Users/glwang/Downloads/6.894/Final_project/MOMA/"+img_id+".jpg"

In [10]:
response = requests.get(data[0]['ThumbnailURL'])
image_id = data[0]['ObjectID']
file_dir = "MOMA/"+str(image_id)+".jpg"
with open(file_dir, 'wb') as f:
    f.write(response.content)

In [10]:
for j in range(1):
    print(j)

0


In [36]:
import extcolors
colors,pixel_count = extcolors.extract(path="/home/olive/BBZY/6.894/2.jpg",tolerance=0)

In [38]:
json.dumps(colors)

'[[[237, 231, 233], 5552], [[236, 230, 232], 4631], [[236, 232, 233], 4114], [[238, 232, 234], 3854], [[235, 231, 232], 3698], [[237, 233, 234], 2887], [[235, 229, 231], 2728], [[234, 230, 231], 2522], [[237, 228, 229], 2118], [[236, 231, 235], 2085], [[238, 229, 230], 1900], [[236, 227, 228], 1888], [[232, 230, 233], 1799], [[236, 230, 230], 1757], [[231, 229, 232], 1703], [[237, 231, 231], 1632], [[239, 233, 235], 1623], [[235, 229, 229], 1553], [[235, 230, 234], 1524], [[233, 229, 230], 1498], [[235, 226, 227], 1436], [[237, 232, 236], 1401], [[238, 234, 235], 1339], [[233, 231, 234], 1325], [[238, 229, 232], 1279], [[234, 228, 230], 1271], [[237, 228, 231], 1217], [[239, 230, 231], 1151], [[230, 228, 231], 1148], [[234, 225, 226], 1132], [[236, 228, 226], 1124], [[237, 229, 227], 1098], [[234, 228, 228], 1020], [[228, 227, 232], 976], [[235, 227, 225], 971], [[239, 230, 233], 965], [[238, 232, 232], 933], [[231, 229, 230], 931], [[227, 226, 231], 915], [[238, 230, 228], 908], [[234

In [38]:
len(a)


5

In [35]:
json.dumps(colors[:5])

'[[[237, 231, 233], 5552], [[236, 230, 232], 4631], [[236, 232, 233], 4114], [[238, 232, 234], 3854], [[235, 231, 232], 3698]]'

In [17]:
import requests
response = requests.get(data[0]['ThumbnailURL'])
if response.status_code == 200:
    colors,pixel_count = extcolors.extract(response.content)


ValueError: embedded null byte

In [ ]:
1 import numpy as np
2 import cv2
3 
4 img = cv2.imread('home.jpg')
5 Z = img.reshape((-1,3))
6 
7 # convert to np.float32
8 Z = np.float32(Z)
9 
10 # define criteria, number of clusters(K) and apply kmeans()
11 criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
12 K = 8
13 ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
14 
15 # Now convert back into uint8, and make original image
16 center = np.uint8(center)
17 res = center[label.flatten()]
18 res2 = res.reshape((img.shape))
19 
20 cv2.imshow('res2',res2)
21 cv2.waitKey(0)
22 cv2.destroyAllWindows()

In [39]:


img = cv2.imread('MOMA/3.jpg')
Z = img.reshape((-1,3))
Z = np.float32(Z)
 
# # define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 8
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# # Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

cv2.imwrite('res2.jpg',res2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

In [50]:
import json

with open('data_s.json', 'r') as data_file:
    jsondata = json.load(data_file)

In [49]:
jsondata[0]

{'Title': 'City of Music, National Superior Conservatory of Music and Dance, Paris, France, View from interior courtyard',
 'Artist': ['Christian de Portzamparc'],
 'ConstituentID': [7470],
 'Date': '1987',
 'Medium': 'Paint and colored pencil on print',
 'Classification': 'Architecture',
 'Department': 'Architecture & Design',
 'ObjectID': 3,
 'URL': 'http://www.moma.org/collection/works/3',
 'ThumbnailURL': 'http://www.moma.org/media/W1siZiIsIjk3Il0sWyJwIiwiY29udmVydCIsIi1yZXNpemUgMzAweDMwMFx1MDAzZSJdXQ.jpg?sha=55b65fa4368fe00a',
 'Height (cm)': 40.6401,
 'Width (cm)': 29.8451,
 'Domain color': 0,
 'Max Color': (186, 164, 140)}

In [48]:
jsondata[0]['Domain color'] = 0

In [ ]:
import numpy as np
import cv2
import requests
import extcolors
import psycopg2
import json
from os import listdir
from os.path import isfile, join

newjson = list()
count  = 0
for item in jsondata:
    img_id = item['ObjectID']  # object ID of the drawing 
    img_dir= join('MOMA',str(img_id)+'.jpg')
#     print(img_dir)
    img = cv2.imread(img_dir)
    Z = img.reshape((-1,3))
    Z = np.float32(Z)

    # # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 8
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    new_img_dir = join('Kmeans_MOMA',img_dir.split('/')[1])
    cv2.imwrite(new_img_dir,res2)
    
    #save the compressed image 
    
    colors,pixel_count = extcolors.extract(path = new_img_dir,tolerance=10)
    colors = [list(f) for f in colors]
#     print(colors)
    channel = min(8,len(colors))
    dominant_color = colors[:channel]
    max_color=colors[0][0]
#     print(dominant_color)
    sum_pixel = sum([x[1] for x in dominant_color])
    for sub in dominant_color:
        sub[1] = sub[1]/sum_pixel
    item['Domain color'] = dominant_color
    item['Max Color']= max_color
    newjson.append(item)
    count = count+1
    
    if count %100 ==0:
        print('100 is done',count)
with open('compress_color_artwork.json', 'w') as outfile:  
    json.dump(newjson, outfile,ensure_ascii=False, indent=4)       
    
# onlyfiles = [join('MOMA',f) for f in listdir('MOMA') ]
# for img_dir in onlyfiles:
#     img = cv2.imread(img_dir)
#     Z = img.reshape((-1,3))
#     Z = np.float32(Z)

#     # # define criteria, number of clusters(K) and apply kmeans()
#     criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
#     K = 8
#     ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

#     # # Now convert back into uint8, and make original image
#     center = np.uint8(center)
#     res = center[label.flatten()]
#     res2 = res.reshape((img.shape))

#     cv2.imwrite(join('Kmeans_MOMA',img_dir.split('/')[1]),res2)


100 is done 100
100 is done 200
100 is done 300
100 is done 400
100 is done 500
100 is done 600
100 is done 700
100 is done 800
100 is done 900
100 is done 1000
100 is done 1100
100 is done 1200
100 is done 1300
100 is done 1400
100 is done 1500
100 is done 1600
100 is done 1700
100 is done 1800
100 is done 1900
100 is done 2000
100 is done 2100
100 is done 2200
100 is done 2300
100 is done 2400
100 is done 2500
100 is done 2600
100 is done 2700
100 is done 2800
100 is done 2900
100 is done 3000
100 is done 3100
100 is done 3200
100 is done 3300
100 is done 3400
100 is done 3500
100 is done 3600
100 is done 3700
100 is done 3800
100 is done 3900
100 is done 4000
100 is done 4100
100 is done 4200
100 is done 4300
100 is done 4400


In [ ]:
onlyfiles[0].split('/')[1].split('.')[0]

In [2]:
!pip install opencv-python

    100% |████████████████████████████████| 26.6MB 2.7MB/s eta 0:00:01
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
#rebuild the img_analysis_table:
imgfiles = [join('Kmeans_MOMA',f) for f in listdir('Kmeans_MOMA') ]
color_dict = dict()

for img_dir in imgfiles:
    img_id = img_dir.split('/')[1].split('.')[0]
    


NameError: name 'json' is not defined

In [9]:


newjson = list()
for item in jsondata:
    img_id = item['ObjectID']  # object ID of the drawing 
    print(index)
    color_list = color_dict[img_id][0]
    sum_pixel = sum([x[1] for x in color_list])
    for sub in color_list:
        sub[1] = sub[1]/sum_pixel
    item['Domain color'] = color_list
    item['Max Color']= color_dict[img_id][1]
    newjson.append(item)

with open('data.json', 'w') as outfile:  
    json.dump(newjson, outfile,ensure_ascii=False, indent=4)        

[((187, 167, 142), 45024),
 ((166, 154, 140), 7443),
 ((127, 115, 99), 3640),
 ((153, 141, 125), 2692),
 ((89, 79, 67), 1975),
 ((229, 222, 206), 1733),
 ((158, 138, 113), 1549),
 ((175, 154, 127), 1394),
 ((132, 114, 90), 305),
 ((181, 168, 151), 252),
 ((94, 79, 60), 198),
 ((222, 223, 215), 70),
 ((236, 220, 197), 14),
 ((167, 161, 139), 4),
 ((153, 146, 120), 3),
 ((190, 170, 135), 2),
 ((97, 77, 50), 1),
 ((163, 154, 149), 1)]